# 1. Import library

In [1]:
# Internal Libraries
import psutil, os, gc, time, csv
from concurrent.futures import ProcessPoolExecutor
import multiprocessing as mp

# External Libraries
import polars as pl
import numpy as np
from tqdm import tqdm

# 2. Import Dataset

In [2]:
# Preaggregation with Polars LazyFrame
trans_lazy = pl.scan_csv('transactions.csv',
    schema_overrides={"value": pl.Float64, "gas": pl.Float64, "gas_price": pl.Float64}
)
# Materialize the LazyFrame into an eager DataFrame
#trans_df = trans_lazy.collect()

# Replace zero flag by -1 flag (0 is good => -1, 1 is bad)
ta_lazy = pl.scan_csv('train_acc.csv').with_columns(pl.col("flag").replace({0: -1}))
ta = ta_lazy.collect()

# For test data, just rename Predict to flag
te_lazy = pl.scan_csv('test_acc_predict.csv').rename({"Predict": "flag"})
te = te_lazy.collect()

# 3. Introduce feature columns

In [3]:
trans_lazy = trans_lazy.with_columns(
    # Calculations are done in a more readable way with preassigned dtype
    pprofit=((pl.col("value").cast(pl.Float64) - pl.col("gas").cast(pl.Float64) * pl.col("gas_price").cast(pl.Float64)) / 1e19),
)

In [4]:
# Define the computation graph
trans_lazy = trans_lazy.with_columns(
    # Calculations are done in a more readable way with preassigned dtype
    profit=pl.when(pl.col("pprofit") > 0).then(pl.col("pprofit")).otherwise(0), #profit means positive profit,
    gas_cost=(pl.col("gas") * pl.col("gas_price")),
    net_value=(pl.col("value") - pl.col("gas") * pl.col("gas_price"))
).with_columns(
    value_to_gas_ratio=(pl.col("value") / (pl.col("gas_cost") + 1e-9)),
    is_zero_value=(pl.col("value") == 0).cast(pl.Int8),
    is_profitable=(pl.col("net_value") > 0).cast(pl.Int8),
).with_columns(
    transaction_time_dt=pl.col("transaction_time_utc").str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S", strict=False)
).with_columns(
    transaction_hour=pl.col("transaction_time_dt").dt.hour().cast(pl.UInt8),
    transaction_day=pl.col("transaction_time_dt").dt.day().cast(pl.UInt8),
    transaction_month=pl.col("transaction_time_dt").dt.month().cast(pl.UInt8),
    transaction_weekday=pl.col("transaction_time_dt").dt.weekday().cast(pl.UInt8),
).with_columns(
    is_weekend=pl.col("transaction_weekday").is_in([5, 6]).cast(pl.UInt8),
    is_night=pl.col("transaction_hour").is_in([0, 1, 2, 3, 4, 5, 22, 23]).cast(pl.UInt8),
    is_morning=pl.col("transaction_hour").is_in([6, 7, 8, 9, 10, 11]).cast(pl.UInt8),
    is_afternoon=pl.col("transaction_hour").is_in([12, 13, 14, 15, 16, 17]).cast(pl.UInt8),
    is_evening=pl.col("transaction_hour").is_in([18, 19, 20, 21]).cast(pl.UInt8)
)

trans = trans_lazy.collect()

# 4. Combine all the accounts and flags

In [ ]:
# Combine account info.
all_accounts_df = pl.concat([ta, te])
tdf = all_accounts_df.with_columns(
    pl.col("flag").cast(pl.Int8).fill_null(0)
).sort("account")

# 5. Functions

## 5.1 Precomputation Dictionary

In [ ]:
print("Pre-computing transaction graph for fast lookups...")

# --- 1. Encode accounts as categorical for compact integer codes ---
trans = trans.with_columns([
    pl.col("from_account").cast(pl.Categorical),
    pl.col("to_account").cast(pl.Categorical)
])

# Integer codes for from/to
from_codes = trans["from_account"].to_physical().to_numpy()
to_codes   = trans["to_account"].to_physical().to_numpy()

# Unique label arrays (same order as codes)
from_uniques = trans["from_account"].cat.get_categories()
to_uniques   = trans["to_account"].cat.get_categories()

# --- 2. Build outgoing edges array ---
edges_out = np.empty(len(trans), dtype=[
    ("neighbor", np.int32),
    ("profit",   np.float64),
    ("pprofit",  np.float64),
    ("transaction_hour", np.uint8),
    ("transaction_day", np.uint8),
    ("transaction_month", np.uint8),
    ("transaction_weekday", np.uint8),
    ("is_weekend", np.uint8),
    ("is_night", np.uint8),
    ("is_morning", np.uint8),
    ("is_afternoon", np.uint8),
    ("is_evening", np.uint8),
    ("gas_cost", np.float64),
    ("net_value", np.float64),
    ("value_to_gas_ratio", np.float64),
    ("is_zero_value", np.uint8),
    ("is_profitable", np.uint8)
])
edges_out["neighbor"] = to_codes
edges_out["profit"]   = trans["profit"].to_numpy()
edges_out["pprofit"]  = trans["pprofit"].to_numpy()
edges_out["transaction_hour"] = trans["transaction_hour"].to_numpy()
edges_out["transaction_day"] = trans["transaction_day"].to_numpy()
edges_out["transaction_month"]	= trans["transaction_month"].to_numpy()
edges_out["transaction_weekday"] = trans["transaction_weekday"].to_numpy()
edges_out["is_weekend"]	= trans["is_weekend"].to_numpy()
edges_out["is_night"] = trans["is_night"].to_numpy()
edges_out["is_morning"]	= trans["is_morning"].to_numpy()
edges_out["is_afternoon"] = trans["is_afternoon"].to_numpy()
edges_out["is_evening"]	= trans["is_evening"].to_numpy()
edges_out["gas_cost"] = trans["gas_cost"].to_numpy()
edges_out["net_value"] = trans["net_value"].to_numpy()
edges_out["value_to_gas_ratio"] = trans["value_to_gas_ratio"].to_numpy()
edges_out["is_zero_value"] = trans["is_zero_value"].to_numpy()
edges_out["is_profitable"] = trans["is_profitable"].to_numpy()

# Sort by from_account code for contiguous slices
order_out = np.argsort(from_codes)
edges_out = edges_out[order_out]
from_codes_sorted = from_codes[order_out]

# Index map: account_code → (start, end)
starts_out = np.searchsorted(from_codes_sorted, np.arange(len(from_uniques)), side="left")
ends_out   = np.searchsorted(from_codes_sorted, np.arange(len(from_uniques)), side="right")
to_neighbors_idx = dict(zip(range(len(from_uniques)), zip(starts_out, ends_out)))

# Map label → code
acc_to_code_out = {acc: i for i, acc in enumerate(from_uniques)}

# --- 3. Build incoming edges array ---
edges_in = np.empty(len(trans), dtype=[
    ("neighbor", np.int32),
    ("profit",   np.float64),
    ("pprofit",  np.float64),
    ("transaction_hour", np.uint8),
    ("transaction_day", np.uint8),
    ("transaction_month", np.uint8),
    ("transaction_weekday", np.uint8),
    ("is_weekend", np.uint8),
    ("is_night", np.uint8),
    ("is_morning", np.uint8),
    ("is_afternoon", np.uint8),
    ("is_evening", np.uint8),
    ("gas_cost", np.float64),
    ("net_value", np.float64),
    ("value_to_gas_ratio", np.float64),
    ("is_zero_value", np.uint8),
    ("is_profitable", np.uint8)
])
edges_in["neighbor"] = from_codes

edges_in["profit"]   = trans["profit"].to_numpy()
edges_in["transaction_hour"] = trans["transaction_hour"].to_numpy()
edges_in["transaction_day"] = trans["transaction_day"].to_numpy()
edges_in["transaction_month"]	= trans["transaction_month"].to_numpy()
edges_in["transaction_weekday"] = trans["transaction_weekday"].to_numpy()
edges_in["is_weekend"]	= trans["is_weekend"].to_numpy()
edges_in["is_night"] = trans["is_night"].to_numpy()
edges_in["is_morning"]	= trans["is_morning"].to_numpy()
edges_in["is_afternoon"] = trans["is_afternoon"].to_numpy()
edges_in["is_evening"]	= trans["is_evening"].to_numpy()
edges_in["gas_cost"] = trans["gas_cost"].to_numpy()
edges_in["net_value"] = trans["net_value"].to_numpy()
edges_in["value_to_gas_ratio"] = trans["value_to_gas_ratio"].to_numpy()
edges_in["is_zero_value"] = trans["is_zero_value"].to_numpy()
edges_in["is_profitable"] = trans["is_profitable"].to_numpy()

# Sort by to_account code for contiguous slices
order_in = np.argsort(to_codes)
edges_in = edges_in[order_in]
to_codes_sorted = to_codes[order_in]

# Index map: account_code → (start, end)
starts_in = np.searchsorted(to_codes_sorted, np.arange(len(to_uniques)), side="left")
ends_in   = np.searchsorted(to_codes_sorted, np.arange(len(to_uniques)), side="right")
from_neighbors_idx = dict(zip(range(len(to_uniques)), zip(starts_in, ends_in)))

# Map label → code
acc_to_code_in = {acc: i for i, acc in enumerate(to_uniques)}

print("Graph pre-computation complete.")
print("Memory loaded.")
print(f"edges_out: {edges_out.nbytes / 1024 / 1024:.2f} MB")
print(f"edges_in:  {edges_in.nbytes  / 1024 / 1024:.2f} MB")

Pre-computing transaction graph for fast lookups...
Graph pre-computation complete.
Memory loaded.
edges_out: 350.07 MB
edges_in:  350.07 MB


In [20]:
edges_out

array([(  6359, 0.00000000e+00, 0.00000000e+00, -1.19965076e-04, 20, 14, 7, 2, 0, 0, 0, 0, 1, 1.19965076e+15, -1.19965076e+15, 0.00000000e+00, 1, 0),
       (     0, 0.00000000e+00, 0.00000000e+00, -1.66945500e-04, 21, 14, 7, 2, 0, 0, 0, 0, 1, 1.66945500e+15, -1.66945500e+15, 0.00000000e+00, 1, 0),
       (  2951, 0.00000000e+00, 0.00000000e+00, -5.66162000e-04, 10, 23, 7, 4, 0, 0, 1, 0, 0, 5.66162000e+15, -5.66162000e+15, 0.00000000e+00, 1, 0),
       ...,
       (   577, 1.98984013e+18, 1.98968026e-01,  1.98968026e-01, 13,  4, 5, 1, 0, 0, 0, 1, 0, 1.59871350e+14,  1.98968026e+18, 1.24465086e+04, 0, 1),
       (419528, 5.90246577e+16, 5.83106577e-03,  5.83106577e-03, 13,  4, 5, 1, 0, 0, 0, 1, 0, 7.14000000e+14,  5.83106577e+16, 8.26675879e+01, 0, 1),
       (419532, 2.00000000e+15, 1.79000000e-04,  1.79000000e-04, 13,  4, 5, 1, 0, 0, 0, 1, 0, 2.10000000e+14,  1.79000000e+15, 9.52380952e+00, 0, 1)],
      shape=(5826604,), dtype=[('neighbor', '<i4'), ('value', '<f8'), ('profit', '<f8')

In [ ]:
data_dict_in = {name: edges_in[name] for name in edges_in.dtype.names}

# Create a Polars DataFrame from the dictionary
df = pl.DataFrame(data_dict_in)

# Write the DataFrame to a CSV file
df.write_csv("edge_in.csv")

In [ ]:
data_dict_out = {name: edges_out[name] for name in edges_out.dtype.names}

# Create a Polars DataFrame from the dictionary
df = pl.DataFrame(data_dict_out)

# Write the DataFrame to a CSV file
df.write_csv("edge_out.csv")

In [9]:
# --- 1. Encode accounts as categorical for compact integer codes ---
trans = trans.with_columns([
    pl.col("from_account").cast(pl.Categorical),
    pl.col("to_account").cast(pl.Categorical)
])

# Unique label arrays (same order as codes)
from_uniques = trans["from_account"].cat.get_categories()
to_uniques   = trans["to_account"].cat.get_categories()

# 1) Build unified node ids
all_accounts = pl.concat([from_uniques.to_frame(name="acc"),
                          to_uniques.to_frame(name="acc")]).unique()
acc_list = all_accounts["acc"].to_list()
acc_to_id = {acc: i for i, acc in enumerate(acc_list)}
id_to_acc = {i: acc for acc, i in acc_to_id.items()}
n_nodes = len(acc_list)

# 2) Convert your transactions to unified integer edges
src_ids = np.array([acc_to_id[a] for a in trans["from_account"].to_list()], dtype=np.int32)
dst_ids = np.array([acc_to_id[a] for a in trans["to_account"].to_list()], dtype=np.int32)

# 3) Build CSR (outgoing) adjacency
order = np.argsort(src_ids)
src_sorted = src_ids[order]
dst_sorted = dst_ids[order]

indptr = np.zeros(n_nodes + 1, dtype=np.int64)
# count degrees
np.add.at(indptr, src_sorted + 1, 1)
# prefix sum
indptr = np.cumsum(indptr)

indices = dst_sorted.astype(np.int32)  # neighbors array length = num_edges

# Optional: if you need incoming CSR as well
order_in = np.argsort(dst_ids)
dst_sorted2 = dst_ids[order_in]
src_sorted2 = src_ids[order_in]
indptr_in = np.zeros(n_nodes + 1, dtype=np.int64)
np.add.at(indptr_in, dst_sorted2 + 1, 1)
indptr_in = np.cumsum(indptr_in)
indices_in = src_sorted2.astype(np.int32)

# Keep your per-edge feature arrays in the same sorted order as 'indices'
# Example for net_value, gas_cost:
edge_net_value = trans["net_value"].to_numpy()[order].astype(np.float32)
edge_gas_cost = trans["gas_cost"].to_numpy()[order].astype(np.float32)

## 5.2 find_to_nei(acc)

Purpose: Finds all accounts that received transactions FROM a given account

In [13]:
to_uniques_list = to_uniques.to_list()  # plain Python list of labels

def find_to_nei(acc):
    code = acc_to_code_out.get(acc)
    if code is None:
        return []
    start, end = to_neighbors_idx[code]
    nei_codes = edges_out["neighbor"][start:end]
    return [
        (to_uniques_list[int(nei)], float(edges_out["profit"][i]), float(edges_out["pprofit"][i]), int(edges_out["transaction_hour"][i]), int(edges_out["transaction_day"][i]), int(edges_out["transaction_month"][i]), int(edges_out["transaction_weekday"][i]), int(edges_out["is_weekend"][i]), int(edges_out["is_night"][i]), int(edges_out["is_morning"][i]), int(edges_out["is_afternoon"][i]), int(edges_out["is_evening"][i]), float(edges_out["gas_cost"][i]), float(edges_out["net_value"][i]), float(edges_out["value_to_gas_ratio"][i]), int(edges_out["is_zero_value"][i]), int(edges_out["is_profitable"][i]))
        for i, nei in enumerate(nei_codes, start)
    ]

In [14]:
#find_to_nei('a29595')

## 5.3 find_from_nei(acc)

Purpose: Finds all forward transaction paths up to depth h starting from account acc

In [15]:
from_uniques_list = from_uniques.to_list()  # plain Python list of labels

def find_from_nei(acc):
    """Incoming neighbors: (neighbor_label, profit, pprofit)"""
    code = acc_to_code_in.get(acc)
    if code is None:
        return []
    start, end = from_neighbors_idx[code]
    nei_codes = edges_in["neighbor"][start:end]
    return [
        (from_uniques[int(nei)], float(edges_in["profit"][i]), float(edges_in["pprofit"][i]), int(edges_in["transaction_hour"][i]), int(edges_in["transaction_day"][i]), int(edges_in["transaction_month"][i]), int(edges_in["transaction_weekday"][i]), int(edges_in["is_weekend"][i]), int(edges_in["is_night"][i]), int(edges_in["is_morning"][i]), int(edges_in["is_afternoon"][i]), int(edges_in["is_evening"][i]), float(edges_in["gas_cost"][i]), float(edges_in["net_value"][i]), float(edges_in["value_to_gas_ratio"][i]), int(edges_in["is_zero_value"][i]), int(edges_in["is_profitable"][i]))
        for i, nei in enumerate(nei_codes, start)
    ]

In [16]:
#find_from_nei('a29595')

## 5.4 find_forward_paths(acc)

In [17]:
def find_forward_paths(acc, h):
    path = [(acc, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0, 0)]  # (account, profit, pprofit, transaction_hour, transaction_day, transaction_month, transaction_weekday, is_weekend, is_night, is_morning, is_afternoon, is_evening, gas_cost, net_value, value_to_gas_ratio, is_zero_value, is_profitable)
    
    def dfs():
        if len(path) == h or not (neighbors := find_to_nei(path[-1][0])):
            yield tuple(path)
            return
        for s, t, a, a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12, a13, a14 in neighbors:
            path.append((s, t, a, a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12, a13, a14))
            yield from dfs()
            path.pop()

    yield from dfs()

In [18]:
#path = [("a111", 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0, 0), ("a1122", 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0, 0)]

In [19]:
#path[-1]

In [20]:
#len(path)

In [ ]:
# find_forward_paths('a29595', 3)

## 5.5 find_backward_paths

In [23]:
def find_backward_paths(acc, h):
    path = [(acc, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0, 0)]  # (account, profit, pprofit, transaction_hour, transaction_day, transaction_month, transaction_weekday, is_weekend, is_night, is_morning, is_afternoon, is_evening, gas_cost, net_value, value_to_gas_ratio, is_zero_value, is_profitable)

    def dfs():
        if len(path) == h or not (neighbors := find_from_nei(path[-1][0])):
            yield tuple(path)
            return
        for s, t, a, a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12, a13, a14 in neighbors:
            path.append((s, t, a, a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12, a13, a14))
            yield from dfs()
            path.pop()

    yield from dfs()

## 5.5 global_flag_map

In [24]:
flag_map = dict(zip(tdf["account"].to_list(), tdf["flag"].to_list()))

In [25]:
GLOBAL_FLAG_MAP = None

def init_globals():
    global GLOBAL_FLAG_MAP
    GLOBAL_FLAG_MAP = dict(zip(tdf["account"].to_list(), tdf["flag"].to_list()))

In [26]:
init_globals()

## 5.6 Combined (extract_features_from_cnt + extract_features_from_cnt2 + find_weights)

In [27]:
import math

class RunningStats:
    """Streaming accumulator for mean, variance, min, max."""
    __slots__ = ("count", "mean", "M2", "min_val", "max_val")

    def __init__(self):
        self.count = 0
        self.mean = 0.0
        self.M2 = 0.0  # sum of squares of differences from the mean
        self.min_val = float("inf")
        self.max_val = float("-inf")

    def update(self, x: float):
        self.count += 1
        delta = x - self.mean
        self.mean += delta / self.count
        delta2 = x - self.mean
        self.M2 += delta * delta2
        self.min_val = min(self.min_val, x)
        self.max_val = max(self.max_val, x)

    @property
    def variance(self):
        return self.M2 / self.count if self.count > 0 else 0.0

    @property
    def std(self):
        return math.sqrt(self.variance)

    def as_dict(self, prefix: str):
        return {
            f"{prefix}_count": self.count,
            f"{prefix}_mean": self.mean if self.count else 0.0,
            f"{prefix}_std": self.std if self.count else 0.0,
            f"{prefix}_min": self.min_val if self.count else 0.0,
            f"{prefix}_max": self.max_val if self.count else 0.0,
        }

In [28]:
def stream_and_aggregate_paths(paths_generator, prefix: str, path_limit: int = 2_000_000_000_000):
    global GLOBAL_FLAG_MAP
    if GLOBAL_FLAG_MAP is None:
        raise RuntimeError("GLOBAL_FLAG_MAP is not initialized. Call init_globals() before using find_weights.")
    """
    Processes a generator of transaction paths to aggregate features on the fly.

    This function is memory-efficient as it processes one path at a time
    instead of loading all paths into memory. It also includes a path limit
    to prevent excessive runtimes for highly connected "super-hub" accounts.

    Args:
        paths_generator: An iterator that yields individual paths.
        prefix (str): A prefix ('f' for forward, 'b' for backward) to label the features.
        path_limit (int): The maximum number of paths to process for this generator.

    Returns:
        dict: A dictionary containing the aggregated feature values.
    """
    # Initialize a dictionary to hold the aggregated feature values.
    # All features start at zero.

    features = {
        f'normal_{prefix}profit': 0.0, f'normal_{prefix}pprofit': 0.0, f'normal_{prefix}size': 0.0,
        f'abnormal_{prefix}profit': 0.0, f'abnormal_{prefix}pprofit': 0.0, f'abnormal_{prefix}size': 0.0,
        f't_{prefix}profit': 0.0, f't_{prefix}pprofit': 0.0, f't_{prefix}size': 0.0,
        f'A_{prefix}profit': 0.0, f'A_{prefix}pprofit': 0.0, f'A_{prefix}size': 0.0,
        f'B_{prefix}profit': 0.0, f'B_{prefix}pprofit': 0.0, f'B_{prefix}size': 0.0,
        f'A_{prefix}is_weekend': 0.0, f'B_{prefix}is_weekend': 0.0, f'normal_{prefix}is_weekend': 0.0, 
        f'abnormal_{prefix}is_weekend': 0.0, f't_{prefix}is_weekend': 0.0,
        f'A_{prefix}is_night': 0.0, f'B_{prefix}is_night': 0.0, f'normal_{prefix}is_night': 0.0,
        f'abnormal_{prefix}is_night': 0.0, f't_{prefix}is_night': 0.0,
        f'A_{prefix}is_morning': 0.0, f'B_{prefix}is_morning': 0.0, f'normal_{prefix}is_morning': 0.0,
        f'abnormal_{prefix}is_morning': 0.0, f't_{prefix}is_morning': 0.0,
        f'A_{prefix}is_afternoon': 0.0, f'B_{prefix}is_afternoon': 0.0, f'normal_{prefix}is_afternoon': 0.0,
        f'abnormal_{prefix}is_afternoon': 0.0, f't_{prefix}is_afternoon': 0.0,
        f'A_{prefix}is_evening': 0.0, f'B_{prefix}is_evening': 0.0, f'normal_{prefix}is_evening': 0.0,
        f'abnormal_{prefix}is_evening': 0.0, f't_{prefix}is_evening': 0.0,
        #f'A_{prefix}gas_cost': 0.0, f'A_{prefix}net_value': 0.0, f'A_{prefix}value_to_gas_ratio': 0.0,
        f'A_{prefix}is_zero_value': 0.0, f'B_{prefix}is_zero_value': 0.0, f'normal_{prefix}is_zero_value': 0.0,
        f'abnormal_{prefix}is_zero_value': 0.0, f't_{prefix}is_zero_value': 0.0,
        f'A_{prefix}is_profitable': 0.0, f'B_{prefix}is_profitable': 0.0, f'normal_{prefix}is_profitable': 0.0,
        f'abnormal_{prefix}is_profitable': 0.0, f't_{prefix}is_profitable': 0.0
    }
    
    # add hour/day/month/weekday slots
    for h in range(0, 24):
        features[f'A_{prefix}hour_{h}'] = 0.0
        features[f'B_{prefix}hour_{h}'] = 0.0
        features[f'normal_{prefix}hour_{h}'] = 0.0
        features[f'abnormal_{prefix}hour_{h}'] = 0.0
        features[f't_{prefix}hour_{h}'] = 0.0
    for d in range(1, 32):
        features[f'A_{prefix}day_{d}'] = 0.0
        features[f'B_{prefix}day_{d}'] = 0.0
        features[f'normal_{prefix}day_{d}'] = 0.0
        features[f'abnormal_{prefix}day_{d}'] = 0.0
        features[f't_{prefix}day_{d}'] = 0.0
    for m in range(1, 13):
        features[f'A_{prefix}month_{m}'] = 0.0
        features[f'B_{prefix}month_{m}'] = 0.0
        features[f'normal_{prefix}month_{m}'] = 0.0
        features[f'abnormal_{prefix}month_{m}'] = 0.0
        features[f't_{prefix}month_{m}'] = 0.0
    for w in range(1, 8):
        features[f'A_{prefix}weekday_{w}'] = 0.0
        features[f'B_{prefix}weekday_{w}'] = 0.0
        features[f'normal_{prefix}weekday_{w}'] = 0.0
        features[f'abnormal_{prefix}weekday_{w}'] = 0.0
        features[f't_{prefix}weekday_{w}'] = 0.0

    # streaming stats accumulators for continuous features
    stats = {
        "A": {
            "gas_cost": RunningStats(),
            "net_value": RunningStats(),
            "value_to_gas_ratio": RunningStats(),
        },
        "B": {
            "gas_cost": RunningStats(),
            "net_value": RunningStats(),
            "value_to_gas_ratio": RunningStats(),
        },
        "normal": {
            "gas_cost": RunningStats(),
            "net_value": RunningStats(),
            "value_to_gas_ratio": RunningStats(),
        },
        "abnormal": {
            "gas_cost": RunningStats(),
            "net_value": RunningStats(),
            "value_to_gas_ratio": RunningStats(),
        },
        "t": {
            "gas_cost": RunningStats(),
            "net_value": RunningStats(),
            "value_to_gas_ratio": RunningStats(),
        },
    }

    path_count = 0
    # Process one path at a time directly from the generator.
    # This is the key to maintaining low memory usage.
    for path in paths_generator:
        # Stop processing if the path limit is reached.
        if path_count >= path_limit:
            print(f"[{prefix.upper()}] Path limit of {path_limit:,} reached. Halting aggregation for this direction.")
            break
        
        path_count += 1
        #print(path)

        if not path:
            continue

        # The features are derived from the terminal node of the path.
        (acc, profit, pprofit, transaction_hour, transaction_day, transaction_month,
         transaction_weekday, is_weekend, is_night, is_morning, is_afternoon, is_evening,
         gas_cost, net_value, value_to_gas_ratio, is_zero_value, is_profitable) = path[-1]
        
        flag = GLOBAL_FLAG_MAP.get(acc)
        if path and all(v == 0 or v == 0.0 for v in path[-1][1:]): #and len(path) <= 1
            if acc.startswith('a'):
                features[f'A_{prefix}profit'] += profit
                features[f'A_{prefix}pprofit'] += pprofit
                features[f'A_{prefix}size'] += 1
                features[f'A_{prefix}is_weekend'] += is_weekend
                features[f'A_{prefix}is_night'] += is_night
                features[f'A_{prefix}is_morning'] += is_morning
                features[f'A_{prefix}is_afternoon'] += is_afternoon
                features[f'A_{prefix}is_evening'] += is_evening
                features[f'A_{prefix}is_zero_value'] += is_zero_value
                features[f'A_{prefix}is_profitable'] += is_profitable
                # update streaming stats
                stats["A"]["gas_cost"].update(gas_cost)
                stats["A"]["net_value"].update(net_value)
                stats["A"]["value_to_gas_ratio"].update(value_to_gas_ratio)
            elif acc.startswith('b'):
                features[f'B_{prefix}profit'] += profit
                features[f'B_{prefix}pprofit'] += pprofit
                features[f'B_{prefix}size'] += 1
                features[f'B_{prefix}is_weekend'] += is_weekend
                features[f'B_{prefix}is_night'] += is_night
                features[f'B_{prefix}is_morning'] += is_morning
                features[f'B_{prefix}is_afternoon'] += is_afternoon
                features[f'B_{prefix}is_evening'] += is_evening
                features[f'B_{prefix}is_zero_value'] += is_zero_value
                features[f'B_{prefix}is_profitable'] += is_profitable
                # update streaming stats
                stats["B"]["gas_cost"].update(gas_cost)
                stats["B"]["net_value"].update(net_value)
                stats["B"]["value_to_gas_ratio"].update(value_to_gas_ratio)
            if flag == -1:  # Flag for 'good'/'normal' accounts
                features[f'normal_{prefix}profit'] += profit
                features[f'normal_{prefix}pprofit'] += pprofit
                features[f'normal_{prefix}size'] += 1
                features[f'normal_{prefix}is_weekend'] += is_weekend
                features[f'normal_{prefix}is_night'] += is_night
                features[f'normal_{prefix}is_morning'] += is_morning
                features[f'normal_{prefix}is_afternoon'] += is_afternoon
                features[f'normal_{prefix}is_evening'] += is_evening
                features[f'normal_{prefix}is_zero_value'] += is_zero_value
                features[f'normal_{prefix}is_profitable'] += is_profitable
                # update streaming stats
                stats["normal"]["gas_cost"].update(gas_cost)
                stats["normal"]["net_value"].update(net_value)
                stats["normal"]["value_to_gas_ratio"].update(value_to_gas_ratio)
            elif flag == 1:  # Flag for 'bad'/'abnormal' accounts (after replacing 1)
                features[f'abnormal_{prefix}profit'] += profit
                features[f'abnormal_{prefix}pprofit'] += pprofit
                features[f'abnormal_{prefix}size'] += 1
                features[f'abnormal_{prefix}is_weekend'] += is_weekend
                features[f'abnormal_{prefix}is_night'] += is_night
                features[f'abnormal_{prefix}is_morning'] += is_morning
                features[f'abnormal_{prefix}is_afternoon'] += is_afternoon
                features[f'abnormal_{prefix}is_evening'] += is_evening
                features[f'abnormal_{prefix}is_zero_value'] += is_zero_value
                features[f'abnormal_{prefix}is_profitable'] += is_profitable
                # update streaming stats
                stats["abnormal"]["gas_cost"].update(gas_cost)
                stats["abnormal"]["net_value"].update(net_value)
                stats["abnormal"]["value_to_gas_ratio"].update(value_to_gas_ratio)
            elif flag == 0: # neither good nor bad (not figured out)
                features[f't_{prefix}profit'] += profit
                features[f't_{prefix}pprofit'] += pprofit
                features[f't_{prefix}size'] += 1
                features[f't_{prefix}is_weekend'] += is_weekend
                features[f't_{prefix}is_night'] += is_night
                features[f't_{prefix}is_morning'] += is_morning
                features[f't_{prefix}is_afternoon'] += is_afternoon
                features[f't_{prefix}is_evening'] += is_evening
                features[f't_{prefix}is_zero_value'] += is_zero_value
                features[f't_{prefix}is_profitable'] += is_profitable
                # update streaming stats
                stats["t"]["gas_cost"].update(gas_cost)
                stats["t"]["net_value"].update(net_value)
                stats["t"]["value_to_gas_ratio"].update(value_to_gas_ratio)
            #print(f"Skipping zeroed path for account {path[0][0]}: {path}")
            continue

        if not (1 <= transaction_day <= 31):
            print(f"⚠️ Unexpected transaction_day={transaction_day} for account {acc}")
        #print(transaction_day)


        # --- Aggregate features based on the terminal account type ('a' or 'b') ---
        if acc.startswith('a'):
            features[f'A_{prefix}profit'] += profit
            features[f'A_{prefix}pprofit'] += pprofit
            features[f'A_{prefix}size'] += 1
            features[f'A_{prefix}hour_{transaction_hour}'] += 1
            features[f'A_{prefix}day_{transaction_day}'] += 1
            features[f'A_{prefix}month_{transaction_month}'] += 1
            features[f'A_{prefix}weekday_{transaction_weekday}'] += 1
            features[f'A_{prefix}is_weekend'] += is_weekend
            features[f'A_{prefix}is_night'] += is_night
            features[f'A_{prefix}is_morning'] += is_morning
            features[f'A_{prefix}is_afternoon'] += is_afternoon
            features[f'A_{prefix}is_evening'] += is_evening
            features[f'A_{prefix}is_zero_value'] += is_zero_value
            features[f'A_{prefix}is_profitable'] += is_profitable

            # update streaming stats
            stats["A"]["gas_cost"].update(gas_cost)
            stats["A"]["net_value"].update(net_value)
            stats["A"]["value_to_gas_ratio"].update(value_to_gas_ratio)


        elif acc.startswith('b'):
            features[f'B_{prefix}profit'] += profit
            features[f'B_{prefix}pprofit'] += pprofit
            features[f'B_{prefix}size'] += 1
            features[f'B_{prefix}hour_{transaction_hour}'] += 1
            features[f'B_{prefix}day_{transaction_day}'] += 1
            features[f'B_{prefix}month_{transaction_month}'] += 1
            features[f'B_{prefix}weekday_{transaction_weekday}'] += 1
            features[f'B_{prefix}is_weekend'] += is_weekend
            features[f'B_{prefix}is_night'] += is_night
            features[f'B_{prefix}is_morning'] += is_morning
            features[f'B_{prefix}is_afternoon'] += is_afternoon
            features[f'B_{prefix}is_evening'] += is_evening
            features[f'B_{prefix}is_zero_value'] += is_zero_value
            features[f'B_{prefix}is_profitable'] += is_profitable

            # update streaming stats
            stats["B"]["gas_cost"].update(gas_cost)
            stats["B"]["net_value"].update(net_value)
            stats["B"]["value_to_gas_ratio"].update(value_to_gas_ratio)

        # --- Aggregate features based on the known flag of the terminal account ---
        # This requires the GLOBAL_FLAG_MAP to be initialized.
        flag = GLOBAL_FLAG_MAP.get(acc)
        if flag == -1:  # Flag for 'good'/'normal' accounts
            features[f'normal_{prefix}profit'] += profit
            features[f'normal_{prefix}pprofit'] += pprofit
            features[f'normal_{prefix}size'] += 1
            features[f'normal_{prefix}hour_{transaction_hour}'] += 1
            features[f'normal_{prefix}day_{transaction_day}'] += 1
            features[f'normal_{prefix}month_{transaction_month}'] += 1
            features[f'normal_{prefix}weekday_{transaction_weekday}'] += 1
            features[f'normal_{prefix}is_weekend'] += is_weekend
            features[f'normal_{prefix}is_night'] += is_night
            features[f'normal_{prefix}is_morning'] += is_morning
            features[f'normal_{prefix}is_afternoon'] += is_afternoon
            features[f'normal_{prefix}is_evening'] += is_evening
            features[f'normal_{prefix}is_zero_value'] += is_zero_value
            features[f'normal_{prefix}is_profitable'] += is_profitable
            # update streaming stats
            stats["normal"]["gas_cost"].update(gas_cost)
            stats["normal"]["net_value"].update(net_value)
            stats["normal"]["value_to_gas_ratio"].update(value_to_gas_ratio)

        elif flag == 1:  # Flag for 'bad'/'abnormal' accounts (after replacing 1)
            features[f'abnormal_{prefix}profit'] += profit
            features[f'abnormal_{prefix}pprofit'] += pprofit
            features[f'abnormal_{prefix}size'] += 1
            features[f'abnormal_{prefix}hour_{transaction_hour}'] += 1
            features[f'abnormal_{prefix}day_{transaction_day}'] += 1
            features[f'abnormal_{prefix}month_{transaction_month}'] += 1
            features[f'abnormal_{prefix}weekday_{transaction_weekday}'] += 1
            features[f'abnormal_{prefix}is_weekend'] += is_weekend
            features[f'abnormal_{prefix}is_night'] += is_night
            features[f'abnormal_{prefix}is_morning'] += is_morning
            features[f'abnormal_{prefix}is_afternoon'] += is_afternoon
            features[f'abnormal_{prefix}is_evening'] += is_evening
            features[f'abnormal_{prefix}is_zero_value'] += is_zero_value
            features[f'abnormal_{prefix}is_profitable'] += is_profitable
            # update streaming stats
            stats["abnormal"]["gas_cost"].update(gas_cost)
            stats["abnormal"]["net_value"].update(net_value)
            stats["abnormal"]["value_to_gas_ratio"].update(value_to_gas_ratio)

        elif flag == 0: # neither good nor bad (not figured out)
            features[f't_{prefix}profit'] += profit
            features[f't_{prefix}pprofit'] += pprofit
            features[f't_{prefix}size'] += 1
            features[f't_{prefix}hour_{transaction_hour}'] += 1
            features[f't_{prefix}day_{transaction_day}'] += 1
            features[f't_{prefix}month_{transaction_month}'] += 1
            features[f't_{prefix}weekday_{transaction_weekday}'] += 1
            features[f't_{prefix}is_weekend'] += is_weekend
            features[f't_{prefix}is_night'] += is_night
            features[f't_{prefix}is_morning'] += is_morning
            features[f't_{prefix}is_afternoon'] += is_afternoon
            features[f't_{prefix}is_evening'] += is_evening
            features[f't_{prefix}is_zero_value'] += is_zero_value
            features[f't_{prefix}is_profitable'] += is_profitable
            # update streaming stats
            stats["t"]["gas_cost"].update(gas_cost)
            stats["t"]["net_value"].update(net_value)
            stats["t"]["value_to_gas_ratio"].update(value_to_gas_ratio)
    
    # flatten streaming stats into features
    for group, group_stats in stats.items():
        for name, rs in group_stats.items():
            features.update(rs.as_dict(f"{group}_{prefix}{name}"))


    return features

In [29]:
#stream_and_aggregate_paths(find_forward_paths('a26056', 3), 'f')

In [30]:
#stream_and_aggregate_paths(find_backward_paths('a26056', 3), 'b')

## 5.9 Prepare account list (25198 in length)

In [31]:
# Extract account columns
ta_acc = ta.select("account")
te_acc = te.select("account")

# Concatenate and drop duplicates
all_accounts_df = pl.concat([ta_acc, te_acc]).unique()

# Convert to list if needed for iteration
all_accounts = all_accounts_df["account"].to_list()

## 5.10 Find heavy accounts

In [32]:
def account_connectivity(acc):
    return len(find_to_nei(acc)) + len(find_from_nei(acc))

In [33]:
connectivity_scores = [(acc, account_connectivity(acc)) for acc in all_accounts]
connectivity_scores.sort(key=lambda x: -x[1])  # descending

In [34]:
heavy_accounts = [acc for acc, score in connectivity_scores[:40]]
light_accounts = [acc for acc, score in connectivity_scores[40:]]

## 5.11 Memory Guard

In [35]:
def ram_ok(threshold_mb=7000):
    mem = psutil.Process().memory_info().rss / 1024 / 1024
    return mem < threshold_mb

# 6. Executor

In [ ]:
def process_account(acc):
    global GLOBAL_FLAG_MAP
    if GLOBAL_FLAG_MAP is None:
        raise RuntimeError("GLOBAL_FLAG_MAP is not initialized. Call init_globals() before using find_weights.")
    """
    Processes a single account to generate its features using a memory-efficient
    streaming approach.
    """
    start_time = time.time()

    # 1. Generate all features for both directions using the efficient streaming function.
    # This is now the ONLY place where path-finding happens.
    forward_features = stream_and_aggregate_paths(find_forward_paths(acc, 3), 'f')
    backward_features = stream_and_aggregate_paths(find_backward_paths(acc, 3), 'b')
    #print(acc, GLOBAL_FLAG_MAP.get(acc))

    # 2. Determine the data category based on whether paths ending in known accounts were found.
    # This replaces the old `has_vv_cnt` logic in a more direct way.
    found_known_forward = forward_features['normal_fsize'] > 0 or forward_features['abnormal_fsize'] > 0
    found_known_backward = backward_features['normal_bsize'] > 0 or backward_features['abnormal_bsize'] > 0

    if found_known_forward and found_known_backward:
        label = 'data1'
    elif found_known_forward:
        label = 'data2'
    elif found_known_backward:
        label = 'data3'
    else:
        label = 'data4'

    # 3. Combine all features into a single dictionary. This ensures that all output
    #    files can have a consistent column structure.
    result_row = {'account': acc, 'flag': flag_map.get(acc, 0)}
    result_row.update(forward_features)
    result_row.update(backward_features)
    
    # 4. Clean up memory.
    del forward_features, backward_features
    gc.collect()

    elapsed = time.time() - start_time
    print(f"[{acc}] Time: {elapsed:.2f}s | Label: {label}")

    return (label, result_row, acc)

# --- Set number of workers based on available cores ---
num_workers = min(20, os.cpu_count() or 1)

# Set up global flag map for worker processes
init_globals()
if GLOBAL_FLAG_MAP is None:
    raise RuntimeError("GLOBAL_FLAG_MAP not initialized")

output_filenames = {
    'data1': 'data1_df.csv',
    'data2': 'data2_df.csv',
    'data3': 'data3_df.csv',
    'data4': 'data4_df.csv',
}

# --- Clean up old output files ---
for f in output_filenames.values():
    if os.path.exists(f):
        os.remove(f)

print(f"🚀 Starting processing with {num_workers} CPU cores...")
print(f"Heavy accounts will run sequentially. Light accounts will run in parallel.")
print(f"Results will be written to: {list(output_filenames.values())} on the fly.")

# They are the file handler and the csv writer for each label
file_handles = {}
csv_writers = {}

def write_result(label, result_row):
    """
    Writes a result row to the correct CSV file efficiently by keeping
    file handles open and deciding intelligently between 'w' and 'a'.
    """
    if not label or not result_row:
        return

    # First time writing for this label
    if label not in csv_writers:
        try:
            filename = output_filenames[label]
            file_exists = os.path.exists(filename) and os.path.getsize(filename) > 0

            # 'w' if file doesn't exist or is empty, else 'a'
            mode = 'a' if file_exists else 'w'
            f_handle = open(filename, mode, newline='', encoding='utf-8')
            file_handles[label] = f_handle

            writer = csv.DictWriter(f_handle, fieldnames=result_row.keys())

            # Only write header if file is new/empty
            if not file_exists:
                writer.writeheader()

            csv_writers[label] = writer

        except KeyError:
            print(f"Error: Label '{label}' not found in output_filenames. Skipping.")
            return

    # Append the row
    csv_writers[label].writerow(result_row)
    file_handles[label].flush()  # Ensure it's written to disk

# IMPORTANT: Remember to keep the loop at the end of your script to close the files:
# for f_handle in file_handles.values():
#     f_handle.close()

log_file = open("details.log", "w", encoding="utf-8")

def heavy_account_worker(acc, q):
    label, result_data, acc = process_account(acc)
    q.put((label, result_data, acc))

🚀 Starting processing with 20 CPU cores...
Heavy accounts will run sequentially. Light accounts will run in parallel.
Results will be written to: ['data1_df.csv', 'data2_df.csv', 'data3_df.csv', 'data4_df.csv'] on the fly.


In [ ]:
if __name__ == "__main__":
    # This block ensures the multiprocessing code runs safely.

    # --- 1. Heavy Computation (Sequential, Isolated Processes) ---
    print("Processing heavy accounts sequentially to conserve RAM...")

    # A multiprocessing Queue is used to safely get the result back from the isolated process.
    result_queue = mp.Queue()

    pbar = tqdm(total=len(heavy_accounts), desc="Heavy Accounts", ncols=80)

    # Process each heavy account in its own dedicated subprocess, one after another.
    for account in heavy_accounts:
        time.sleep(3)
        p = mp.Process(target=heavy_account_worker, args=(account, result_queue))
        p.start()
        p.join()

        if not result_queue.empty():
            label, result_row, acc = result_queue.get()

            # Log details to file instead of printing to console
            log_file.write(f"[{acc}] Label: {label} | Data: {result_row}\n")
            log_file.flush()

            filename = output_filenames[label]
            file_exists = os.path.exists(filename) and os.path.getsize(filename) > 0
            with open(filename, "a", newline="", encoding="utf-8") as f:
                writer = csv.DictWriter(f, fieldnames=result_row.keys())
                if not file_exists:
                    writer.writeheader()
                writer.writerow(result_row)
                f.flush()

        # Move progress bar forward by 1
        pbar.update(1)

    pbar.close()

    # --- 2. Light Computation (Parallel Process Pool) ---
    print(f"\nProcessing light accounts in parallel using {num_workers} workers...")

    # A ProcessPoolExecutor is used to manage a pool of worker processes for parallel execution.
    # The `initializer` ensures each worker process has the global flag map available.
    with ProcessPoolExecutor(max_workers=num_workers, initializer=init_globals) as executor:
        results_iterator = executor.map(process_account, light_accounts)

        for label, result_row, acc in tqdm(results_iterator, total=len(light_accounts), desc="Light Accounts", ncols=80):
            # Log to the same file
            log_file.write(f"[{acc}] Label: {label} | Data: {result_row}\n") # not being fixed
            log_file.flush()

            # Write to CSV directly inside the loop
            if label and result_row:
                filename = output_filenames.get(label)
                if not filename:
                    print(f"⚠️ Label '{label}' not found in output_filenames.")
                    continue

                file_exists = os.path.exists(filename) and os.path.getsize(filename) > 0

                try:
                    with open(filename, "a", newline="", encoding="utf-8") as f:
                        writer = csv.DictWriter(f, fieldnames=result_row.keys())
                        if not file_exists:
                            writer.writeheader()
                        writer.writerow(result_row)
                        f.flush()
                except Exception as e:
                    print(f"❌ Failed to write result for label '{label}': {e}")
        log_file.close()



    # --- 3. Finalization ---
    print("\nProcessing complete. Closing all output files...")
    # Cleanly close all file handles that were opened for writing results.
    for f_handle in file_handles.values():
        f_handle.close()

    print("All tasks finished successfully.")

Processing heavy accounts sequentially to conserve RAM...


Heavy Accounts:   0%|                                    | 0/40 [00:00<?, ?it/s]

[a29595] Time: 1.91s | Label: data2


Heavy Accounts:   2%|▋                           | 1/40 [00:05<03:17,  5.05s/it]

[a09531] Time: 1.22s | Label: data1


...

[a04645] Time: 0.14s | Label: data2[a26328] Time: 0.16s | Label: data3[a22275] Time: 0.15s | Label: data2

[a25063] Time: 0.16s | Label: data2
[a22265] Time: 0.15s | Label: data4

[a03433] Time: 0.16s | Label: data4[a24881] Time: 0.15s | Label: data2

[a00568] Time: 0.16s | Label: data3[a17960] Time: 0.16s | Label: data2

[a28660] Time: 0.14s | Label: data4[a24666] Time: 0.15s | Label: data4

[a17381] Time: 0.16s | Label: data4
[a04223] Time: 0.15s | Label: data1
[a15520] Time: 0.13s | Label: data1
[a06090] Time: 0.12s | Label: data4
[a22036] Time: 0.11s | Label: data3
[a24461] Time: 0.11s | Label: data3
[a02686] Time: 0.10s | Label: data3[a10885] Time: 0.11s | Label: data2[a26544] Time: 0.10s | Label: data1

Light Accounts: 100%|███████████████████▉| 25154/25158 [12:40<00:00, 155.42it/s]




[a00082] Time: 0.09s | Label: data3
[a11364] Time: 0.09s | Label: data1


Light Accounts: 100%|█████████████████████| 25158/25158 [12:40<00:00, 33.08it/s]



Processing complete. Closing all output files...
All tasks finished successfully.
